In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [145]:
def double_exponential_smooth(d, extra_periods = 1, alpha = 0.4, beta = 0.4):
    #length of input list
    cols = len(d) 
   
    for i in range(extra_periods):
        d.append(np.nan)

    #fill forecast, level, and trend columns with nan
    f = np.array([np.nan]*(cols+extra_periods))
    a = np.array([np.nan]*(cols+extra_periods))
    b = np.array([np.nan]*(cols+extra_periods))

    #initialize level and trend columns
    a[0] = d[0]
    b[0] = d[1]-d[0]
    
    #populate level and trend columns
    for i in range(1,cols):
        a[i] = alpha * d[i] + (1-alpha)*(a[i-1] + b[i-1])
        b[i] = beta * (a[i] - a[i-1]) + (1-beta)*b[i-1]

    #forecast next to historical data
    for i in range(1,cols+1):
        f[i] = a[i-1] + b[i-1]

    #forecast into the future
    for i in range(cols+1, extra_periods + cols):
        f[i] = f[i-1] + b[cols-1]



    df = pd.DataFrame.from_dict({'Demand':d, 'Forecast':f, 'Level':a, 'Trend':b})
    df['Error'] = df['Forecast'] - df['Demand']
    return(df)
    
    

In [144]:
def kpi(df):
    #average demand
    dem_ave = df.loc[df['Error'].notnull(), 'Demand'].mean()
    #absolute bias
    bias_abs = df['Error'].mean()
    #relative bias
    bias_rel = bias_abs / dem_ave
    #print results 
    print('Bias: {:0.2f}, {:.2%}'.format(bias_abs, bias_rel))

    #calculate MAPE
    MAPE = (df['Error'].abs() / df['Demand']).mean()
    #print results
    print('MAPE: {:0.2%}'.format(MAPE))

    #calculate absoltue and scaled MAE
    MAE_abs = df['Error'].abs().mean()
    MAE_rel = MAE_abs / dem_ave
    #print results
    print('MAE: {:0.2f}, {:0.2%}'.format(MAE_abs,MAE_rel))

    #calculate absolute and scaled RSME
    RMSE_abs = np.sqrt((df['Error']**2).mean())
    RMSE_rel =  RMSE_abs / dem_ave
    #print results
    print('RMSE: {:0.2f}, {:0.2%}'.format(RMSE_abs,RMSE_rel))

In [146]:
d = [37,60,85,112,132,145,179,198,150,132]
table = double_exponential_smooth(d,5,alpha=0.3,beta=0.4)
table

,Demand,Forecast,Level,Trend,Error
0,37.0,NaN,37.000000,23.000000,NaN
1,60.0,60.000000,60.000000,23.000000,0.000000
2,85.0,83.000000,83.600000,23.240000,-2.000000
3,112.0,106.840000,108.388000,23.859200,-5.160000
4,132.0,132.247200,132.173040,23.829536,0.247200
5,145.0,156.002576,152.701803,22.509227,11.002576
6,179.0,175.211030,176.347721,22.963903,-3.788970
7,198.0,199.311624,198.918137,22.806508,1.311624
8,150.0,221.724645,200.207252,14.199551,71.724645
9,132.0,214.406803,189.684762,4.310735,82.406803


In [147]:
kpi(table)

Bias: 17.30, 13.05%
MAPE: 14.20%
MAE: 19.74, 14.89%
RMSE: 36.67, 27.66%
